# The ATE evaluation

counting rational tokens and classify them by semantic/ non-semantinc for either programming and natural language

In [238]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools
import json
import nltk
import re

pd.options.display.float_format = '{:.2f}'.format

## --------------- Reading form previous tagged processed files ------------

This section is the manual script construction to obtain the semantic/ non-semantinc counts in a manual process

In [4]:
import json
import pandas as pd

In [12]:
def param_default():
    return {
        'dataset' : 'code_completion_random_cut_5k_30_512_tokens',
        #'dataset' : 'code_completion_docstring_random_cut_3.8k_30_150_tokens',
        #'dataset' : 'code_completion_docstring_signature_3.8k_30_150_tokens',
        #'dataset' : 'code_completion_docstring_5k_30_150_tokens',
        'rational_results': '/workspaces/code-rationales/data/rationales/gpt',
        'tagged_rationales': '/workspaces/code-rationales/data/tagged_rationales',
        'delimiter_sequence': '', ## VERY IMPOETANT
        'num_samples' : 100, 
        'size_samples' : 44,
        'num_experiments': 30,
        'model_name' : '/workspaces/code-rationales/data/codeparrot-small/checkpoints/checkpoint-29000', 
        'cache_dir': '/workspaces/code-rationales/datax/df_cache_dir',
        'galeras_path': '/workspaces/code-rationales/semeru-datasets/semeru/galeras/code_rationales/'
    }
params = param_default()

In [15]:
path_tagged = '{}/{}{}{}.csv'.format(params['tagged_rationales'], params['dataset'],'_exp_' , str(0))
path_dataset = '{}{}.json'.format(params['galeras_path'],params['dataset'])
tagged_results = pd.read_csv(path_tagged)
with open(path_dataset) as json_file:
    galeras_dataset = json.load(json_file)


In [18]:
galeras_dataset = pd.DataFrame.from_dict(galeras_dataset)

In [19]:
galeras_dataset.head()

,id,commit_id,repo,path,file_name,fun_name,commit_message,code,url,language,...,n_whitespaces,n_words,vocab_size,complexity,nloc,token_counts,n_ast_nodes,n_identifiers,signature,prompt
0,159555,f9437064b3501869f5f56fb9e6d345d81ffeec5e,rasa,tests/core/test_evaluation.py,test_evaluation.py,skip_on_CI,Skip e2e tests on CI: these tests take too lon...,def skip_on_CI() -> bool:\n \n return os...,https://github.com/RasaHQ/rasa.git,Python,...,18,12,12,1,3,28,55,5,def skip_on_CI() -> bool,def skip_on_CI() -> bool:\n \n return os...
1,276430,84afc5193d38057e2e2badf9c889ea87d80d8fbf,keras,keras/tests/custom_training_loop_test.py,custom_training_loop_test.py,call,Reformatting the codebase with black.\n\nPiper...,"def call(self, inputs):\n self.add_loss...",https://github.com/keras-team/keras.git,Python,...,21,8,8,1,3,24,38,7,"def call(self, inputs)","def call(self, inputs):\n self.add_loss..."
2,210956,39531637b5675a36409c303db022bfab90939896,PaddleDetection,deploy/python/det_keypoint_unite_infer.py,det_keypoint_unite_infer.py,smoothing_factor,one euro filter and ema smoothing for keypoint...,"def smoothing_factor(self, te, fc):\n r...",https://github.com/PaddlePaddle/PaddleDetectio...,Python,...,36,19,16,1,3,28,42,7,"def smoothing_factor(self, te, fc)","def smoothing_factor(self, te, fc):\n r..."
3,190542,4fc3616712edb19179b17dd270ad6cf63abf99c2,DeOldify,fastai/vision/image.py,image.py,data,Upgrading to support latest Pytorch version,"def data(self)->TensorImage:\n ""Return ...",https://github.com/jantic/DeOldify.git,Python,...,24,11,11,1,3,12,22,4,def data(self)->TensorImage,"def data(self)->TensorImage:\n ""Return ..."
4,189667,e040bcacd38378386749db18aeba575b93f4ebca,manim,manim/mobject/geometry/arc.py,arc.py,stop_angle,Improved structure of the :mod:`.mobject` modu...,def stop_angle(self):\n return angle_of...,https://github.com/ManimCommunity/manim.git,Python,...,14,8,8,1,2,24,39,6,def stop_angle(self),def stop_angle(self):\n return angle_of...


In [6]:
tagged_results

,rational_pos,goal_token,from_seq_id,typesets_tgt,rationale_pos_tgt,rationale_prob_tgt,exp,token_type,span,tags
0,0,def,0,NaN,NaN,NaN,0,src,"(0, 3)","['module', 'function_definition', 'def']"
1,1,skip,0,"[('def', 0.00029721998726017773)]",[0],[0.00029721998726017773],0,src,"(3, 8)","['module', 'function_definition']"
2,2,_,0,"[(' skip', 0.3496493101119995)]",[1],[0.3496493101119995],0,src,"(8, 9)","['NN', 'module', 'function_definition', 'ident..."
3,3,on,0,"[('_', 0.001134732156060636), (' skip', 0.0059...","[2, 1, 0]","[0.001134732156060636, 0.0059091681614518166, ...",0,src,"(9, 11)","['NN', 'module', 'function_definition', 'ident..."
4,4,_,0,"[('on', 0.2272750586271286)]",[3],[0.2272750586271286],0,src,"(11, 12)","['NN', 'module', 'function_definition', 'ident..."
...,...,...,...,...,...,...,...,...,...,...
4395,39,(',99,"[('override', 0.030208399519324303), (' import...","[38, 14, 33, 35, 2]","[0.030208399519324303, 0.08234483003616333, 0....",0,src,"(164, 166)","['module', 'function_definition', 'block', 'ER..."
4396,40,pip,99,"[(""('"", 0.00017135975940618664), ('pip', 0.240...","[39, 16]","[0.00017135975940618664, 0.24006126821041107]",0,src,"(166, 169)","['NNS', 'module', 'function_definition', 'bloc..."
4397,41,"',",99,"[('pip', 0.024009736254811287), (""('"", 0.13641...","[40, 39, 38]","[0.024009736254811287, 0.13641567528247833, 0....",0,src,"(169, 171)","['module', 'function_definition', 'block', 'ER..."
4398,42,',99,"[(""',"", 0.31870272755622864)]",[41],[0.31870272755622864],0,src,"(171, 173)","['module', 'function_definition', 'block', 'ER..."


In [21]:
tagged_results.head()

,rational_pos,goal_token,from_seq_id,typesets_tgt,rationale_pos_tgt,rationale_prob_tgt,exp,token_type,span,tags,tag_count
0,0,def,0,NaN,NaN,NaN,0,src,"(0, 3)","['module', 'function_definition', 'def']",40
1,1,skip,0,"[('def', 0.00029721998726017773)]",[0],[0.00029721998726017773],0,src,"(3, 8)","['module', 'function_definition']",33
2,2,_,0,"[(' skip', 0.3496493101119995)]",[1],[0.3496493101119995],0,src,"(8, 9)","['NN', 'module', 'function_definition', 'ident...",53
3,3,on,0,"[('_', 0.001134732156060636), (' skip', 0.0059...","[2, 1, 0]","[0.001134732156060636, 0.0059091681614518166, ...",0,src,"(9, 11)","['NN', 'module', 'function_definition', 'ident...",53
4,4,_,0,"[('on', 0.2272750586271286)]",[3],[0.2272750586271286],0,src,"(11, 12)","['NN', 'module', 'function_definition', 'ident...",53


In [48]:
import re

In [94]:
def convert_string_to_array(input):
    pattern = "(',\s+')|(\')"
    x = re.sub(pattern," ",input)
    return x.split(" ")[1:-1]

In [39]:
def get_keys_by_value(values, target):
    keys = []
    for key, value in values.items():
        if target in value:
            keys.append(key)
    return keys

In [121]:
from collections import Counter

In [119]:
def tag_classification(tags) -> dict:
    sc_dict = pl_taxonomy_python()
    nl_dict = nl_pos_taxonomy()
    categories = list()
    semantic_dic = global_groups()
    semantic_non_semantic = list()
    for tag in tags:
        sc_keys = get_keys_by_value(sc_dict,tag)
        nl_keys = get_keys_by_value(nl_dict,tag)
        if sc_keys:
            categories.extend(sc_keys)
        if nl_keys:
            categories.extend(nl_keys)
        if not sc_keys and not nl_keys:
            categories.append('unknown')
    for category in categories:
        semantic_non_semantic.extend(get_keys_by_value(semantic_dic,category))
    frequency_count = Counter(semantic_non_semantic)
    result = dict(frequency_count)
    return result
    

In [95]:
tagged_results['tag_array'] = tagged_results['tags'].apply(lambda x: convert_string_to_array(x))

In [142]:
tagged_results.head()

,rational_pos,goal_token,from_seq_id,typesets_tgt,rationale_pos_tgt,rationale_prob_tgt,exp,token_type,span,tags,tag_count,tag_array,tag_semantic_array
0,0,def,0,NaN,NaN,NaN,0,src,"(0, 3)","['module', 'function_definition', 'def']",None,"[module, function_definition, def]","{'unknown': 1, 'sc_semantic': 2}"
1,1,skip,0,"[('def', 0.00029721998726017773)]",[0],[0.00029721998726017773],0,src,"(3, 8)","['module', 'function_definition']",None,"[module, function_definition]","{'unknown': 1, 'sc_semantic': 1}"
2,2,_,0,"[(' skip', 0.3496493101119995)]",[1],[0.3496493101119995],0,src,"(8, 9)","['NN', 'module', 'function_definition', 'ident...",None,"[NN, module, function_definition, identifier]","{'nl_semantic': 1, 'unknown': 1, 'sc_semantic'..."
3,3,on,0,"[('_', 0.001134732156060636), (' skip', 0.0059...","[2, 1, 0]","[0.001134732156060636, 0.0059091681614518166, ...",0,src,"(9, 11)","['NN', 'module', 'function_definition', 'ident...",None,"[NN, module, function_definition, identifier]","{'nl_semantic': 1, 'unknown': 1, 'sc_semantic'..."
4,4,_,0,"[('on', 0.2272750586271286)]",[3],[0.2272750586271286],0,src,"(11, 12)","['NN', 'module', 'function_definition', 'ident...",None,"[NN, module, function_definition, identifier]","{'nl_semantic': 1, 'unknown': 1, 'sc_semantic'..."


In [143]:
tagged_results['tag_semantic_array'] = tagged_results['tag_array'].apply(lambda x: tag_classification(x))
tagged_results['tag_count'] = tagged_results['tag_array'].apply(lambda x: len(x))

In [125]:
def sum_counters(counter_list):
    result_counter = Counter()
    for counter in counter_list:
        result_counter += counter
    return result_counter

In [130]:
taggeg_grouped = tagged_results.groupby('from_seq_id')['tag_semantic_array'].apply(sum_counters).reset_index()

In [131]:
taggeg_grouped

,from_seq_id,level_1,tag_semantic_array
0,0,unknown,46.0
1,0,sc_semantic,94.0
2,0,nl_semantic,20.0
3,0,sc_nl,21.0
4,0,sc_not_semantic,51.0
...,...,...,...
695,99,nl_semantic,26.0
696,99,sc_nl,26.0
697,99,sc_not_semantic,8.0
698,99,nl_not_semantic,5.0


In [134]:
tagged_grouped = taggeg_grouped.pivot(index='from_seq_id', columns='level_1', values='tag_semantic_array').reset_index()


In [145]:
tagged_total_counts = tagged_results.groupby('from_seq_id')['tag_count'].sum().reset_index()

In [138]:
selected_galeras = galeras_dataset.head(100)

In [147]:
result_df = pd.concat([selected_galeras,tagged_grouped, tagged_total_counts.iloc[:,1] ], axis=1)

In [151]:
result_df

,id,commit_id,repo,path,file_name,fun_name,commit_message,code,url,language,...,prompt,from_seq_id,nl_not_semantic,nl_semantic,sc_errors,sc_nl,sc_not_semantic,sc_semantic,unknown,tag_count
0,159555,f9437064b3501869f5f56fb9e6d345d81ffeec5e,rasa,tests/core/test_evaluation.py,test_evaluation.py,skip_on_CI,Skip e2e tests on CI: these tests take too lon...,def skip_on_CI() -> bool:\n \n return os...,https://github.com/RasaHQ/rasa.git,Python,...,def skip_on_CI() -> bool:\n \n return os...,0,6.0,20.0,9.0,21.0,51.0,94.0,46.0,240
1,276430,84afc5193d38057e2e2badf9c889ea87d80d8fbf,keras,keras/tests/custom_training_loop_test.py,custom_training_loop_test.py,call,Reformatting the codebase with black.\n\nPiper...,"def call(self, inputs):\n self.add_loss...",https://github.com/keras-team/keras.git,Python,...,"def call(self, inputs):\n self.add_loss...",1,14.0,22.0,44.0,22.0,76.0,47.0,48.0,258
2,210956,39531637b5675a36409c303db022bfab90939896,PaddleDetection,deploy/python/det_keypoint_unite_infer.py,det_keypoint_unite_infer.py,smoothing_factor,one euro filter and ema smoothing for keypoint...,"def smoothing_factor(self, te, fc):\n r...",https://github.com/PaddlePaddle/PaddleDetectio...,Python,...,"def smoothing_factor(self, te, fc):\n r...",2,9.0,20.0,44.0,20.0,41.0,63.0,53.0,240
3,190542,4fc3616712edb19179b17dd270ad6cf63abf99c2,DeOldify,fastai/vision/image.py,image.py,data,Upgrading to support latest Pytorch version,"def data(self)->TensorImage:\n ""Return ...",https://github.com/jantic/DeOldify.git,Python,...,"def data(self)->TensorImage:\n ""Return ...",3,8.0,20.0,19.0,24.0,14.0,61.0,51.0,189
4,189667,e040bcacd38378386749db18aeba575b93f4ebca,manim,manim/mobject/geometry/arc.py,arc.py,stop_angle,Improved structure of the :mod:`.mobject` modu...,def stop_angle(self):\n return angle_of...,https://github.com/ManimCommunity/manim.git,Python,...,def stop_angle(self):\n return angle_of...,4,8.0,23.0,5.0,23.0,63.0,139.0,46.0,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,158193,b64b41d8c1ac23c43f7a4e3f9f6339d6f0012ab2,d2l-zh,d2l/mxnet.py,mxnet.py,forward,[PaddlePaddle] Merge master into Paddle branch...,"def forward(self, X):\n # `X` shape: (b...",https://github.com/d2l-ai/d2l-zh.git,Python,...,"def forward(self, X):\n # `X` shape: (b...",95,15.0,20.0,44.0,28.0,14.0,7.0,48.0,163
96,289189,bbe63bca4733b31b6f6cf29270cdd62765309fdf,core,homeassistant/components/switchbot/sensor.py,sensor.py,native_value,Bump pySwitchbot to 0.20.0 for bleak 0.19 chan...,def native_value(self) -> str | int | None:\n ...,https://github.com/home-assistant/core.git,Python,...,def native_value(self) -> str | int | None:\n ...,96,10.0,17.0,5.0,17.0,56.0,110.0,48.0,252
97,189952,ce9acddd41f39929d499b45e52621a2849b8333e,manim,tests/test_graphical_units/test_threed.py,test_threed.py,test_Cone,Improved performance of ``test_threed.py`` (#2...,def test_Cone(scene):\n scene.add(Cone(reso...,https://github.com/ManimCommunity/manim.git,Python,...,def test_Cone(scene):\n scene.add(Cone(reso...,97,8.0,23.0,22.0,23.0,43.0,66.0,69.0,245
98,161543,c80ad26d872efe19a8b8eca022489c241f10b9e2,rich,benchmarks/benchmarks.py,benchmarks.py,time_render_unicode_heavy,Add initial benchmark suite,def time_render_unicode_heavy(self):\n ...,https://github.com/Textualize/rich.git,Python,...,def time_render_unicode_heavy(self):\n ...,98,10.0,27.0,1.0,27.0,64.0,168.0,46.0,332


In [150]:
result_df.describe()

,id,n_ast_errors,ast_levels,n_whitespaces,n_words,vocab_size,complexity,nloc,token_counts,n_ast_nodes,n_identifiers,from_seq_id,nl_not_semantic,nl_semantic,sc_errors,sc_nl,sc_not_semantic,sc_semantic,unknown,tag_count
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,71.000000,100.000000,100.00000,100.000000,100.00000,100.000000
mean,182748.120000,0.050000,8.460000,19.030000,7.810000,7.580000,1.270000,4.870000,26.120000,33.640000,5.120000,49.500000,8.820000,21.200000,15.788732,23.370000,29.02000,94.330000,55.00000,233.500000
std,97359.286552,0.261116,1.209558,6.653707,2.762282,2.391821,1.052702,6.559217,35.117201,7.967763,1.320239,29.011492,2.637645,3.874287,14.224642,5.892394,14.49206,35.740664,13.37569,37.334821
min,284.000000,0.000000,5.000000,5.000000,3.000000,3.000000,1.000000,2.000000,8.000000,17.000000,2.000000,0.000000,2.000000,14.000000,1.000000,14.000000,6.00000,7.000000,5.00000,140.000000
25%,118701.750000,0.000000,8.000000,14.000000,6.000000,6.000000,1.000000,2.000000,15.000000,28.750000,4.000000,24.750000,7.000000,19.000000,5.500000,20.000000,18.75000,66.750000,48.00000,210.750000
50%,188258.500000,0.000000,8.000000,20.000000,7.500000,7.000000,1.000000,3.000000,18.000000,32.500000,5.000000,49.500000,9.000000,21.000000,12.000000,23.000000,26.50000,93.500000,50.00000,233.000000
75%,277977.250000,0.000000,9.000000,23.000000,9.000000,9.000000,1.000000,4.000000,24.000000,39.000000,6.000000,74.250000,10.000000,24.000000,18.500000,26.000000,35.25000,121.500000,57.50000,254.250000
max,334797.000000,2.000000,12.000000,36.000000,19.000000,16.000000,8.000000,34.000000,274.000000,64.000000,9.000000,99.000000,17.000000,32.000000,62.000000,55.000000,76.00000,186.000000,99.00000,332.000000


## --- Optimizing code for multiple trials ---

This is the automatic AVG counting among all trials, the result is the AVG matrix per each rationale group for each input prompt. Each row corresponds to each prompt with the calculated confounders

In [239]:

class Parameters():
    def __init__(self) -> None:
        self.datasets = ['code_completion_random_cut_5k_30_512_tokens','code_completion_docstring_random_cut_3.8k_30_150_tokens']
        self.rational_results_path = '/workspaces/code-rationales/data/rationales/gpt'
        self.tagged_rationales_path = '/workspaces/code-rationales/data/tagged_rationales'
        self.num_samples = 100
        self.num_trials = 30
        self.cache_dir = '/workspaces/code-rationales/datax/evaluation/'
        self.save_dir = '/workspaces/code-rationales/datax/evaluation/ETA'
        self.confounders_path = '/workspaces/code-rationales/semeru-datasets/semeru/galeras/code_rationales/'  # This is the galeras reference
        
    def sc_taxonomy_python(self) -> dict:
        return {
            "punctuation": ['{', '}', '[', ']', '(', ')','\"', ',', '.', '...', ';', ':'], 
            "exceptions": ['raise_statement','catch', 'try', 'finally', 'throw', 'throws', 'except'],
            "oop": ['def','class','instanceof','interface','private','protected','public','abstract','extends','package','this','implements','import','new','super'],
            "asserts": ['assert'],
            "types": ['tuple','set','list','pair','subscript','type','none','dictionary','integer','native','static','synchronized','transient','volatile','void','final','enum','byte','char','float','boolean','double','int','long','short','strictfp'],
            "conditionals": ['else', 'if', 'switch', 'case', 'default'],
            "loops": ['break', 'do', 'for', 'while', 'continue'],
            "operators": ['as','yield','is','@','in','and','or','not','**','slice','%','+','<','>','=','+','-','*','/','%','++','--','!','==','!=','>=','<=','&&','||','?',':','~','<<','>>','>>>','&','^','|','//'],
            "indentation": ['\n','\t'],
            "bool": ['true', 'false'], 
            "functional":['lambda','lambda_parameters'],
            "with" : ['with','with_item','with_statement','with_clause'], 
            "return" :['return'],
            "structural" : ['attribute', 'argument_list','parenthesized_expression','pattern_list','class_definition','function_definition','block'],
            "statements" : ['return_statement','break_statement','assignment','while_statement','expression_statement','assert_statement'],
            "expression": ['call','exec','async','ellipsis','unary_operator','binary_operator','as_pattern_target','boolean_operator','as_pattern','comparison_operator','conditional_expression','named_expression','not_operator','primary_expression','as_pattern'],
            "errors": ["ERROR"],
            "identifier":["identifier"],  
            "comment":["comment"],
            "string": ['string','interpolation','string_content','string_end','string_start','escape_sequence'], 
            "unknown": []
        }
        
    def nl_pos_taxonomy(self) -> dict: 
        return {
            "nl_verb" : ['VBN', 'VBG', 'VBZ', 'VBP', 'VBD', 'VB'],
            "nl_noun" : ['NN', 'NNPS', 'NNS', 'NNP'],
            "nl_pronoun" : ['WP', 'PRP', 'PRP$', 'WP','WP$'], 
            "nl_adverb" : ['RBS','RBR', 'RB', 'WRB'], 
            "nl_adjetive" : ['JJR', 'JJS', 'JJ'], 
            "nl_determier" : ['DT','WDT','PDT'], 
            "nl_preposition" : ['IN', 'TO'],
            "nl_particle" : ['RP'],
            "nl_modal" : ['MD'],
            "nl_conjunction" : ['CC'],
            "nl_cardinal" : ['CD'],
            "nl_list": ['LS'],
            "nl_other" : ['FW', 'EX', 'SYM' , 'UH', 'POS', "''", '--',':', '(', ')', '.', ',', '``', '$']
        }
        
    def global_groups(self) -> dict:
        return {
            'sc_semantic': ['exceptions', 'oop', 'asserts', 'types', 'conditionals', 'loops', 'bool', 'structural', 'statements', 'with'], 
            'sc_nl': ['identifier', 'comment', 'string'],
            'sc_not_semantic': ['punctuation', 'operators', 'indentation', 'functional', 'return', 'expression'], ## TODO why unkown is here? 
            'sc_errors' : ['errors'], 
            'nl_semantic': ['nl_verb', 'nl_noun', 'nl_pronoun', 'nl_adjetive'],
            'nl_not_semantic' : ['nl_adverb', 'nl_determier', 'nl_preposition', 'nl_particle', 'nl_modal', 'nl_conjunction', 'nl_cardinal', 'nl_list', 'nl_other'],
            'unknown': ['unknown',''] ## NOTE DRC: I added this
        }

In [240]:
from collections import Counter
import re
class Util():
    def __init__(self,args:Parameters) -> None:
        self.sc_taxonomy = args.sc_taxonomy_python()
        self.nl_taxonomy = args.nl_pos_taxonomy()
        self.global_groups = args.global_groups()
        self.complete_taxonomy = {**self.sc_taxonomy, **self.nl_taxonomy}
        self.args = args

        
    def convert_string_to_array(self, input):
        pattern = "(',\s+')|(\')"
        x = re.sub(pattern," ",input)
        return x.split(" ")[1:-1]
    
    def get_keys_by_value(values, target):
        keys = []
        for key, value in values.items():
            if target in value:
                keys.append(key)
        return keys
    
    def tag_classification(self, tags) -> dict:
        
        categories = list()
        semantic_non_semantic = list()
        for tag in tags:
            keys = get_keys_by_value(self.complete_taxonomy,tag)
            if keys:
                categories.extend(keys)
            else:
                categories.append('unknown')
        for category in categories:
            semantic_non_semantic.extend(get_keys_by_value(self.global_groups,category))
        frequency_count = Counter(semantic_non_semantic)
        result = dict(frequency_count)
        return result
    
    def get_tagged_dataset_path(self, dataset, trial=0, file_extension = 'csv'):
        path_tagged = '{}/{}{}{}.{}'.format(self.args.tagged_rationales_path, 
                                            dataset,'_exp_' , str(trial),file_extension)
        return path_tagged
        
        
    def sum_counters(self,counter_list):
        result_counter = Counter()
        for counter in counter_list:
            result_counter += counter
        return result_counter
    
    def load_galeras_dataset(self, dataset):
        path_dataset = '{}{}.json'.format(self.args.confounders_path,dataset)
        with open(path_dataset) as json_file:
            return  json.load(json_file)
        
    def append_confounders(self,avg_df, dataset):
        galeras_dataset = pd.DataFrame.from_dict(self.load_galeras_dataset(dataset))
        selected_galeras = galeras_dataset.head(self.args.num_samples) ## DRC: be aware! we are picking the first samples always from galeras
        return pd.concat([selected_galeras,avg_df], axis=1)
    
    def save__chekpoint_results(self, df:pd.DataFrame, name):
        path = '{}{}_ETA.parquet'.format(self.args.save_dir,name)
        df.to_parquet(path)
    
        

In [241]:
class ETAEvaluator():
    def __init__(self, args:Parameters, dataset:str):
        self.args = args
        self.util = Util(args)
        self.dataset = dataset
    
    def get_tags_count_for_trial(self, trial_number =0)-> pd.DataFrame:
        util = self.util
        tagged_results = pd.read_csv(util.get_tagged_dataset_path(self.dataset,trial_number))
        tagged_results['tag_array'] = tagged_results['tags'].apply(lambda x: util.convert_string_to_array(x))
        tagged_results['tag_semantic_array'] = tagged_results['tag_array'].apply(lambda x: util.tag_classification(x))
        tagged_results['tag_count'] = tagged_results['tag_array'].apply(lambda x: len(x))
        taggeg_grouped = tagged_results.groupby('from_seq_id')['tag_semantic_array'].apply(util.sum_counters).reset_index()
        tagged_grouped = taggeg_grouped.pivot(index='from_seq_id', columns='level_1', values='tag_semantic_array').reset_index()
        tagged_total_counts = tagged_results.groupby('from_seq_id')['tag_count'].sum().reset_index()
        return pd.concat([tagged_grouped, tagged_total_counts.iloc[:,1] ], axis=1)
    
    def calculate_tags_avg_per_samples(self):
        trial_dfs = list()
        for i in range(self.args.num_trials):
            trial = self.get_tags_count_for_trial(i)
            trial_dfs.append(trial)
        concatenated_df = pd.concat(trial_dfs, axis=0)
        row_avg = concatenated_df.groupby('from_seq_id').agg({'nl_not_semantic': 'mean', 'nl_semantic': 'mean',
                                                              'sc_errors': 'mean', 'sc_nl': 'mean',
                                                              'sc_not_semantic': 'mean', 'sc_semantic': 'mean',
                                                              'unknown': 'mean', 'tag_count': 'mean'}).reset_index()
        return row_avg
    
    

    

In [242]:

## Run ETA for all experiments

args = Parameters()
random_cut_dataset = args.datasets[0] # ['code_completion_random_cut_5k_30_512_tokens','code_completion_docstring_random_cut_3.8k_30_150_tokens']
eta_evaluator = ETAEvaluator(args, random_cut_dataset)

In [243]:
avg_df = eta_evaluator.calculate_tags_avg_per_samples()

In [244]:
result_df = eta_evaluator.util.append_confounders(avg_df,random_cut_dataset)
result_df

,id,commit_id,repo,path,file_name,fun_name,commit_message,code,url,language,...,prompt,from_seq_id,nl_not_semantic,nl_semantic,sc_errors,sc_nl,sc_not_semantic,sc_semantic,unknown,tag_count
0,159555,f9437064b3501869f5f56fb9e6d345d81ffeec5e,rasa,tests/core/test_evaluation.py,test_evaluation.py,skip_on_CI,Skip e2e tests on CI: these tests take too lon...,def skip_on_CI() -> bool:\n \n return os...,https://github.com/RasaHQ/rasa.git,Python,...,def skip_on_CI() -> bool:\n \n return os...,0,8.37,18.90,6.70,21.20,40.57,87.43,49.50,221.67
1,276430,84afc5193d38057e2e2badf9c889ea87d80d8fbf,keras,keras/tests/custom_training_loop_test.py,custom_training_loop_test.py,call,Reformatting the codebase with black.\n\nPiper...,"def call(self, inputs):\n self.add_loss...",https://github.com/keras-team/keras.git,Python,...,"def call(self, inputs):\n self.add_loss...",1,13.47,21.40,16.08,21.80,50.67,109.00,46.90,261.40
2,210956,39531637b5675a36409c303db022bfab90939896,PaddleDetection,deploy/python/det_keypoint_unite_infer.py,det_keypoint_unite_infer.py,smoothing_factor,one euro filter and ema smoothing for keypoint...,"def smoothing_factor(self, te, fc):\n r...",https://github.com/PaddlePaddle/PaddleDetectio...,Python,...,"def smoothing_factor(self, te, fc):\n r...",2,8.87,19.60,29.09,19.73,44.80,89.37,48.17,243.07
3,190542,4fc3616712edb19179b17dd270ad6cf63abf99c2,DeOldify,fastai/vision/image.py,image.py,data,Upgrading to support latest Pytorch version,"def data(self)->TensorImage:\n ""Return ...",https://github.com/jantic/DeOldify.git,Python,...,"def data(self)->TensorImage:\n ""Return ...",3,10.50,20.10,11.50,24.67,19.83,79.47,52.30,206.67
4,189667,e040bcacd38378386749db18aeba575b93f4ebca,manim,manim/mobject/geometry/arc.py,arc.py,stop_angle,Improved structure of the :mod:`.mobject` modu...,def stop_angle(self):\n return angle_of...,https://github.com/ManimCommunity/manim.git,Python,...,def stop_angle(self):\n return angle_of...,4,7.50,22.17,16.60,22.40,62.93,109.87,47.20,277.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,158193,b64b41d8c1ac23c43f7a4e3f9f6339d6f0012ab2,d2l-zh,d2l/mxnet.py,mxnet.py,forward,[PaddlePaddle] Merge master into Paddle branch...,"def forward(self, X):\n # `X` shape: (b...",https://github.com/d2l-ai/d2l-zh.git,Python,...,"def forward(self, X):\n # `X` shape: (b...",95,15.37,19.03,11.47,28.20,19.93,68.70,49.60,195.07
96,289189,bbe63bca4733b31b6f6cf29270cdd62765309fdf,core,homeassistant/components/switchbot/sensor.py,sensor.py,native_value,Bump pySwitchbot to 0.20.0 for bleak 0.19 chan...,def native_value(self) -> str | int | None:\n ...,https://github.com/home-assistant/core.git,Python,...,def native_value(self) -> str | int | None:\n ...,96,9.23,17.77,15.16,19.13,35.17,73.00,53.63,207.37
97,189952,ce9acddd41f39929d499b45e52621a2849b8333e,manim,tests/test_graphical_units/test_threed.py,test_threed.py,test_Cone,Improved performance of ``test_threed.py`` (#2...,def test_Cone(scene):\n scene.add(Cone(reso...,https://github.com/ManimCommunity/manim.git,Python,...,def test_Cone(scene):\n scene.add(Cone(reso...,97,8.60,22.77,22.43,22.77,43.27,68.83,72.87,250.37
98,161543,c80ad26d872efe19a8b8eca022489c241f10b9e2,rich,benchmarks/benchmarks.py,benchmarks.py,time_render_unicode_heavy,Add initial benchmark suite,def time_render_unicode_heavy(self):\n ...,https://github.com/Textualize/rich.git,Python,...,def time_render_unicode_heavy(self):\n ...,98,9.67,28.23,13.87,28.37,44.37,130.43,46.80,287.83


In [246]:
eta_evaluator.util.save__chekpoint_results(result_df,random_cut_dataset)